In [ ]:
!nvidia-smi

Thu Feb 25 02:15:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

In [ ]:
!pip install torch

In [ ]:
!pip install nvidia-pyindex

In [ ]:
!pip install --extra-index-url https://pypi.ngc.nvidia.com nvidia-tensorrt 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
import tensorrt as trt 

In [ ]:
!pip install pycuda

In [ ]:
import pycuda

In [ ]:
import torch
from torch.utils.data import TensorDataset
from transformers import BertModel, BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
model = BertModel.from_pretrained("bert-base-uncased")

In [ ]:
texts = ["Hello, my dog is cute", "Go to the store", "get me my bag", "you are a sick person"]*8

In [ ]:
texts

['Hello, my dog is cute',
 'Go to the store',
 'get me my bag',
 'you are a sick person',
 'Hello, my dog is cute',
 'Go to the store',
 'get me my bag',
 'you are a sick person',
 'Hello, my dog is cute',
 'Go to the store',
 'get me my bag',
 'you are a sick person',
 'Hello, my dog is cute',
 'Go to the store',
 'get me my bag',
 'you are a sick person',
 'Hello, my dog is cute',
 'Go to the store',
 'get me my bag',
 'you are a sick person',
 'Hello, my dog is cute',
 'Go to the store',
 'get me my bag',
 'you are a sick person',
 'Hello, my dog is cute',
 'Go to the store',
 'get me my bag',
 'you are a sick person',
 'Hello, my dog is cute',
 'Go to the store',
 'get me my bag',
 'you are a sick person']

In [ ]:
inputs = tokenizer(texts, return_tensors="pt", padding=True) 

In [ ]:
inputs

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102],
        [  101,  2175,  2000,  1996,  3573,   102,     0,     0],
        [  101,  2131,  2033,  2026,  4524,   102,     0,     0],
        [  101,  2017,  2024,  1037,  5305,  2711,   102,     0],
        [  101,  7592,  1010,  2026,  3899,  2003, 10140,   102],
        [  101,  2175,  2000,  1996,  3573,   102,     0,     0],
        [  101,  2131,  2033,  2026,  4524,   102,     0,     0],
        [  101,  2017,  2024,  1037,  5305,  2711,   102,     0],
        [  101,  7592,  1010,  2026,  3899,  2003, 10140,   102],
        [  101,  2175,  2000,  1996,  3573,   102,     0,     0],
        [  101,  2131,  2033,  2026,  4524,   102,     0,     0],
        [  101,  2017,  2024,  1037,  5305,  2711,   102,     0],
        [  101,  7592,  1010,  2026,  3899,  2003, 10140,   102],
        [  101,  2175,  2000,  1996,  3573,   102,     0,     0],
        [  101,  2131,  2033,  2026,  4524,   102,     0,     

In [ ]:
batch_inputs = TensorDataset(inputs["input_ids"], inputs["token_type_ids"], inputs["attention_mask"]).tensors

In [ ]:
BATCH_SIZE = 32
OPSET_VERSION=11
USE_FP16 = True

In [ ]:
from transformers import convert_graph_to_onnx

In [ ]:
input_names = list(inputs.keys())

In [ ]:
outputs = model(**inputs)

In [ ]:

outputs.last_hidden_state.shape

torch.Size([32, 8, 768])

In [ ]:
outputs.pooler_output.shape

torch.Size([32, 768])

In [ ]:
input_names

['input_ids', 'token_type_ids', 'attention_mask']

In [ ]:
torch.onnx.export(model, 
                  args=batch_inputs, 
                  f="bert_base_model.onnx", 
                  input_names=input_names, 
                  output_names=["output"], 
                  export_params=True, 
                  opset_version=OPSET_VERSION)

/usr/local/lib/python3.7/dist-packages/transformers/models/bert/modeling_bert.py:194: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  position_ids = self.position_ids[:, past_key_values_length : seq_length + past_key_values_length]
/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1760: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  input_tensor.shape[chunk_dim] == tensor_shape for input_tensor in input_tensors


In [ ]:
import tensorrt

In [ ]:
## Not useful at all. Fuck you Nvidia!!!
if USE_FP16:
    !trtexec --onnx=trt_model.onnx --saveEngine=bert_engine_pytorch.trt  --explicitBatch --fp16
else:
    !trtexec --onnx=trt_model.onnx --saveEngine=bert_engine_pytorch.trt  --explicitBatch

/bin/bash: trtexec: command not found


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [ ]:
!sudo apt-get install python-libnvinfer-dev python3-libnvinfer-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-libnvinfer-dev is already the newest version (7.2.2-1+cuda11.1).
python-libnvinfer-dev is already the newest version (7.2.2-1+cuda11.1).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


In [ ]:
!dpkg -l | grep TensorRT

ii  libnvinfer-dev                         7.2.2-1+cuda11.1                    amd64        TensorRT development libraries and headers
ii  libnvinfer-plugin-dev                  7.2.2-1+cuda11.1                    amd64        TensorRT plugin libraries
ii  libnvinfer-plugin7                     7.2.2-1+cuda11.1                    amd64        TensorRT plugin libraries
ii  libnvinfer7                            7.2.2-1+cuda11.1                    amd64        TensorRT runtime libraries
ii  libnvonnxparsers-dev                   7.2.2-1+cuda11.1                    amd64        TensorRT ONNX libraries
ii  libnvonnxparsers7                      7.2.2-1+cuda11.1                    amd64        TensorRT ONNX libraries
ii  libnvparsers-dev                       7.2.2-1+cuda11.1                    amd64        TensorRT parsers libraries
ii  libnvparsers7                          7.2.2-1+cuda11.1                    amd64        TensorRT parsers libraries
ii  python-libnvinfer                   

In [ ]:
!echo $LD_LIBRARY_PATH

/usr/lib64-nvidia


In [ ]:
!pip install --upgrade onnxruntime

Requirement already up-to-date: onnxruntime in /usr/local/lib/python3.7/dist-packages (1.6.0)


In [ ]:
import onnxruntime

In [ ]:
sess_options = onnxruntime.SessionOptions()

In [ ]:
sess_options.optimized_model_filepath = "bert_base_model-optimized.onnx"

In [ ]:
session = onnxruntime.InferenceSession("bert_base_model.onnx", sess_options)

In [ ]:
optimized_model_path = "bert_base_model-optimized.onnx"

In [ ]:
import onnx
from onnxruntime.quantization import QuantizationMode, quantize, quantize_dynamic, quantize_static, CalibrationDataReader

In [ ]:
quantized_model_path = "bert_base_model-quantized.onnx"

In [ ]:
# onnx_model = onnx.load(optimized_model_path)
# quantized_model = quantize(model=onnx_model, 
#                            quantization_mode=QuantizationMode.IntegerOps, 
#                            force_fusions=True,
#                            symmetric_weight=True)
# onnx.save_model(quantized_model, quantized_model_path)

dynamic_quantized_model = quantize_dynamic(model_input=optimized_model_path,
                                          model_output=quantized_model_path)


In [ ]:
import numpy as np
numpy_array_inputs = np.array([v.numpy() for k, v in inputs.items()])

In [ ]:
inference_session = onnxruntime.InferenceSession(quantized_model_path)

In [ ]:
numpy_array_dict = {k: v.numpy() for k, v in inputs.items()}

In [ ]:
numpy_array_dict

{'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 

In [ ]:
embeddings = inference_session.run(None, numpy_array_dict)

In [ ]:
embeddings

[array([[[-3.73735428e-02,  2.19441831e-01,  1.93563104e-01, ...,
          -1.01970941e-01,  4.23016697e-02,  3.49321395e-01],
         [ 1.55111864e-01,  2.67533004e-01,  3.20281059e-01, ...,
          -1.09486125e-01,  6.74381182e-02, -9.47885066e-02],
         [-4.19184774e-01,  6.57595158e-01,  5.91633260e-01, ...,
          -7.98321068e-01,  1.05481707e-01,  2.12848887e-01],
         ...,
         [-1.64352700e-01, -4.45064753e-02,  5.54163694e-01, ...,
          -7.70599723e-01, -3.09484452e-01,  5.37495017e-01],
         [-5.30717850e-01,  2.50040628e-02,  2.85957187e-01, ...,
          -3.42200309e-01,  4.57981229e-02, -2.32771441e-01],
         [ 6.56033456e-01, -2.67751813e-02, -2.71166950e-01, ...,
           6.78134859e-01, -5.91480613e-01, -9.90796089e-02]],
 
        [[ 7.66473934e-02, -1.12734348e-01,  4.03019816e-01, ...,
          -1.95155088e-02, -5.76877668e-02,  4.12166476e-01],
         [-1.79752365e-01, -6.61922634e-01,  4.79029834e-01, ...,
          -8.52199420

In [ ]:
ort_input_ids = onnxruntime.OrtValue.ortvalue_from_numpy(inputs["input_ids"].numpy(), device_type="cpu", device_id=0)
ort_token_type_ids = onnxruntime.OrtValue.ortvalue_from_numpy(inputs["token_type_ids"].numpy(), device_type="cpu", device_id=0)
ort_attention_mask = onnxruntime.OrtValue.ortvalue_from_numpy(inputs["attention_mask"].numpy(), device_type="cpu", device_id=0)

io_binding = session.io_binding()

# OnnxRuntime will copy the data over to the CUDA device if it is consumed by nodes on the CUDA device
io_binding.bind_ortvalue_input('input_ids', ort_input_ids)
io_binding.bind_ortvalue_input('token_type_ids', ort_token_type_ids)
io_binding.bind_ortvalue_input('attention_mask', ort_attention_mask)

Y_ort_value = onnxruntime.OrtValue.ortvalue_from_shape_and_type([32, 8, 768], np.float32, 'cpu', 0)
io_binding.bind_ortvalue_output('output', Y_ort_value)

session.run_with_iobinding(io_binding)
Y_embeddings = io_binding.copy_outputs_to_cpu()[0]


In [ ]:
Y_embeddings

array([[[ 0.06407244,  0.5132997 ,  0.06468916, ..., -0.5676768 ,
          0.11633667,  0.29131466],
        [ 0.16641109,  0.40584326,  0.5065138 , ..., -0.3532385 ,
          0.19185555,  0.02976981],
        [-0.47325447,  1.0550802 ,  0.32723206, ..., -0.7967647 ,
          0.35662922, -0.00331088],
        ...,
        [-0.33917296, -0.0660683 ,  0.5086327 , ..., -0.7612312 ,
         -0.27832684,  0.6653786 ],
        [-0.5493221 ,  0.14455572,  0.15016676, ..., -0.42107478,
          0.1896516 ,  0.01410751],
        [ 0.64637595,  0.07127587, -0.45367277, ..., -0.06630263,
         -0.4371574 , -0.17445263]],

       [[ 0.11069668, -0.08293285,  0.42706603, ...,  0.00120921,
          0.25974396,  0.30370587],
        [-0.28896758, -0.6415334 ,  0.62492603, ..., -0.08302327,
         -0.34951532,  0.32458925],
        [-0.47081953, -0.62868   ,  0.3432132 , ..., -0.28400627,
         -0.20963076,  0.27866927],
        ...,
        [ 0.7307848 , -0.04877289, -0.5014908 , ..., -

In [ ]:
!pip install netron

     |████████████████████████████████| 1.4MB 18.7MB/s 


In [ ]:
import netron

In [ ]:
netron.start(file=quantized_model_path)

Serving 'bert_base_model-quantized.onnx' at http://localhost:8081


('localhost', 8081)